# Visualization: Trading Session

In [8]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

alt.data_transformers.enable('csv')
# alt.data_transformers.enable("notebook")

DataTransformerRegistry.enable('csv')

### 1. Define parameters and Load model

In [9]:
from trading_bot.agent import Agent

model_name = 'model_debug_50'
test_stock = 'data/NQ=F.csv'
window_size = 15
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

OSError: Unable to open file (unable to open file: name = 'models/model_debug_50', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

### 2. Load test data

In [ ]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Adj Close']]
# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2024-12-09 00:00:00,21637.50
1,2024-12-09 00:01:00,21640.00
2,2024-12-09 00:02:00,21639.75
3,2024-12-09 00:03:00,21639.50
4,2024-12-09 00:04:00,21639.50


### 3. Running Eval

In [ ]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install() # level='DEBUG'
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1][3] - test_data[0][3]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)
total_profit = test_result

2024-12-22 11:46:27 DESKTOP-5IBS3KG root[13672] INFO nqf_test_3_10: +$3275.00



### 4. Visualize

In [5]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0][3]] + [x[0][3] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions


    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)

    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [ ]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)

In [7]:
print(total_profit)

3275
